In [1]:
import sys
sys.path.append("/Users/rossg/Work/nanoparticles/pipeline")
#from openmoltools import openeye         # for Choderalab wrappers of openeye software
import openeye_mod as openeye             # personal modification of antechamber file generation function
import openeye.oechem as oechem
from openmoltools import packmol
import parmed as pmd
import subprocess
import os

# Preparation via tleap

Creating a set of functions to automate the setup.

In [3]:
def parametrize_mol(inname,resname):
    """
    Parameterizes molecule with antechamber and adds charges using Open Eye's am1-bcc method from a mol2 file.
    
    Parameters
    ----------
    inname  : str
      the names of the mol2 file of the molecule
    resname : str
      the name you wish to assign as the residue name 
      
    Returns
    -------
    nothing
    
    BUT 'resname'_gaff.mol2 and 'resname'.frcmod are created in the working directory.
    """
    mol = oechem.OEGraphMol()           # initialising molecule object
    ifs = oechem.oemolistream()         # initialising input stream for reading in data
    ifs.SetFormat(oechem.OEFormat_MOL2) # specifying mol2 format input
    ifs.open(inname)           
    if oechem.OEReadMolecule(ifs,mol):  # this function automatically returns True or False, to help spot for errors. 
        pass
    else:
        print "Problem loading molecule!"
    mol = oechem.OEMol(mol)    
    oechem.OEAssignAromaticFlags(mol, oechem.OEAroModelOpenEye)    
    oechem.OEAddExplicitHydrogens(mol)
    mol.SetTitle(resname)
    if any([atom.GetName() == '' for atom in mol.GetAtoms()]):
        oechem.OETriposAtomNames(mol)
    openeye.oemol_to_antechamber(mol,resname+'_gaff.mol2',resname+'.frcmod',residue_name=resname)
    
def mol2pdb(inname):
    outname = inname.split('.')[0] + '.pdb'
    parm = pmd.load_file(inname,structure=False)
    parm.save(outname,overwrite=True)

def oemol2pdb(mol2name,pdbname):
    mol = oechem.OEGraphMol()           # initialising molecule object
    ifs = oechem.oemolistream()         # initialising INPUT stream for reading in data
    ofs = oechem.oemolostream()         # initialising the OUTPUT stream for writing data
    # Reading:
    ifs.SetFormat(oechem.OEFormat_MOL2) # specifying mol2 format input
    ifs.open(mol2name)           
    if oechem.OEReadMolecule(ifs,mol):
        pass
    else:
        print "Problem loading molecule!"
    # Writing:
    ofs.SetFormat(oechem.OEFormat_PDB)      # specifying that I want a pdb format 
    ofs.open(pdbname)
    oechem.OEWriteMolecule(ofs,mol)    
    
    
def tleap_nanosim(drugname,dyename,boxname,solvate=True):
    tleap_input = """
    source leaprc.ff99SB
    source leaprc.gaff
    {0} = loadmol2 {0}_gaff.mol2
    {1} = loadmol2 {1}_gaff.mol2
    loadamberparams {0}.frcmod
    loadamberparams {1}.frcmod
    {2} = loadpdb {2}.pdb
    solvateBox {2} TIP3PBOX 0.5
    addIons2 box Na+ 0
    saveamberparm {2} out.prmtop out.inpcrd
    quit
    """.format(drugname,dyename,boxname)
    file_handle = open('tleap_commands', 'w')
    file_handle.writelines(tleap_input)
    file_handle.close()

    cmd = "tleap -f {0}".format(file_handle.name)
    subprocess.call(cmd,shell=True)
    
def tleap_mixture(resnames,boxname,solvate=True,implicit=False,run=True):
    """
    Creates and runs tleap commands for setting up mixture simulations of small molecules. 
    The mixture can be solvated with water.
    
    Parameters
    ----------
    resnames  : list of str
      the names of the molecules in the mixture. Must match residue name and antechamber mol2 file.
      i.e. resname[1]= 'LIG', implies the residue name is LIG and there is an antechamber mol2 file 'LIG_gaff.mol2'.
    boxname : str
      the name of pdb file (without extension .pdb) file that contains the mixture of molecules in resnames
    solvate = bool
      whether to fill the free space in boxname.pdb with TIP3P water. 
      The box is neutralized assuming there is a net negative charge.
    run: bool
      whether to run tleap
      
    Returns
    -------
    tleap_input : str
      the input file used for tleap
    """
    tleap_input = "source leaprc.ff99SB\nsource leaprc.gaff\n"
    for name in resnames: 
        tleap_input += "{0} = loadmol2 {0}_gaff.mol2\n".format(name)
        tleap_input += "loadamberparams {0}.frcmod\n".format(name)
    tleap_input += "{0} = loadpdb {0}.pdb\n".format(boxname)
    if solvate == True and implicit == False:
        tleap_input += "solvateBox box TIP3PBOX 0.5\n"
        #tleap_input += "addIons2 box Na+ 0\n"       # Not necessary if ions are already added by packmol
    elif solvate == True and implicit == True:
        tleap_input += "set default PBRadii mbondi2\n"  # Set for IGB=5 radii
    elif implicit == True:
        tleap_input += "set default PBRadii mbondi2\n"  # Set for IGB=5 radii
    tleap_input += "saveamberparm {0} out.prmtop out.inpcrd\nquit".format(boxname)
    if run == True:
        file_handle = open('tleap_commands', 'w')
        file_handle.writelines(tleap_input)
        file_handle.close()
        cmd = "tleap -f {0}".format(file_handle.name)
        subprocess.call(cmd,shell=True)
    return tleap_input
    
def nanopipeline(drugmol2,dyemol2,drugname='LIG',dyename='DYE',ndrugs=10,ndye=20,boxsize=50,neutralize=True,implicit=False):
    """
    Starting with the mol2 files of drug and dye, the molecules are parametrised, assembled uniformly in 
    cubic box, and AMBER prmtop and inpcrd files are created for the system. 
    
    Parameters
    ----------
    drugmol2, dyemol2  : str, str
      the mol2 file name of the drug/dye, with or without hydrogens
    drugname, dyename  : str, str
      the residue name you want for the drug/dye residue name and output file naming schemes
    ndrugs/ndye: int
      the number of drug/dye molecules you want in the box
    boxsize: int
      the length of the cubic box in Angstroms
      
    Returns
    -------
    out.inpcrd and out.out.prmtop are created in the working directory, as well gaff output files
    """
    
    # Parmetrizing with antechamber via openeye:
    parametrize_mol(drugmol2,drugname)
    parametrize_mol(dyemol2,dyename)
    # Creating box of drugs and dye. Packmol requires PDB files
    mol2pdb(drugname+'_gaff.mol2')
    mol2pdb(dyename+'_gaff.mol2')
    if neutralize == True:
        box = packmol.pack_box([drugname+'_gaff.pdb',dyename+'_gaff.pdb','Na.pdb'],[ndrugs,ndye,ndye], tolerance=2.0, box_size=boxsize)
    else:
        box = packmol.pack_box([drugname+'_gaff.pdb',dyename+'_gaff.pdb'],[ndrugs,ndye], tolerance=2.0, box_size=boxsize)
    box.save('box.pdb')
    #os.remove(drugname+'_gaff.pdb')
    #os.remove(dyename+'_gaff.pdb')
    # Running tleap to create out.prmtop out.inpcrd for use with openmm
    #tleap_nanosim(drugname,dyename,'box')
    if implicit == False:
        tleap_mixture([drugname,dyename],'box',solvate=True,implicit=False,run=True)
    else:
        tleap_mixture([drugname,dyename],'box',solvate=False,implicit=True,run=True)
                            
def mixture_setup(molfiles,resnames,nmols,boxsize,solvate=True):
    """
    Function that generalises the function 'nanopipeline' for general mixtures.
    Starting with the mol2 files of drug and dye, the molecules are parametrised, assembled uniformly in 
    cubic box, and AMBER prmtop and inpcrd files are created for the system. 
    
    Parameters
    ----------
    molfiles : list of str
      the mol2 file name of the compounds, with or without hydrogens
    resnames : list of str 
      the residue names you want for the molecules  residue name and output file naming schemes
    nmols : list of int
      the number of molecules you want in the box, excluding water and ions
    boxsize: int
      the length of the cubic box in Angstroms
      
    Returns
    -------
    out.inpcrd and out.out.prmtop are created in the working directory, as well gaff output files
    """
    
    # Parmetrizing with antechamber via openeye:
    for i in range(len(molfiles)): parametrize_mol(molfiles[i],resnames[i])
    # Creating box of drugs and dye. Packmol requires PDB files
    for i in range(len(molfiles)): mol2pdb(resnames[i]+'_gaff.mol2')
    molfiles_pdb = [mol + '_gaff.pdb' for mol in resnames]                   
    box = packmol.pack_box(molfiles_pdb, nmols, tolerance=2.0, box_size=boxsize)
    box.save('box.pdb')
    # Running tleap to create out.prmtop out.inpcrd for use with openmm
    tleap_input = tleap_mixture(resnames,'box',solvate=solvate,run=True)

In [4]:
#nanopipeline('1E.mol2','IR783.mol2',ndrugs=5,ndye=2,boxsize=30,neutralize=False)
mixture_setup(['1E.mol2','IR783.mol2'],['1ED','DYE'], [1,1],15,solvate=False)


# Mixture 

tolerance 2.000000
filetype pdb
output /var/folders/_2/bwj1fkzx78v92gc1lfvf8cv589gk45/T/tmpCteGuC.pdb
add_amber_ter


structure 1ED_gaff.pdb
  number 1 
  inside box 0. 0. 0. 15.000000 15.000000 15.000000
end structure

structure DYE_gaff.pdb
  number 1 
  inside box 0. 0. 0. 15.000000 15.000000 15.000000
end structure


# Mixture 

tolerance 2.000000
filetype pdb
output /var/folders/_2/bwj1fkzx78v92gc1lfvf8cv589gk45/T/tmpCteGuC.pdb
add_amber_ter


structure 1ED_gaff.pdb
  number 1 
  inside box 0. 0. 0. 15.000000 15.000000 15.000000
end structure

structure DYE_gaff.pdb
  number 1 
  inside box 0. 0. 0. 15.000000 15.000000 15.000000
end structure



# Preparation without tleap

In [2]:
from simtk.openmm import app, unit
from simtk import openmm
import sys
sys.path.append("/Users/rossg/software/openmoltools")
from openmoltools.forcefield_generators import gaffTemplateGenerator
from openeye import oechem             # the whole of the openeye toolbox

First, we need a way to create PDB from our molecules so that `packmol` can stuff our system for us. As we have MOL2 files, the function below turns these into PDB files (with CONECT records) using `OpenEye`.

In [3]:
def oemol2pdb(mol2name,pdbname):
    mol = oechem.OEGraphMol()           # initialising molecule object
    ifs = oechem.oemolistream()         # initialising INPUT stream for reading in data
    ofs = oechem.oemolostream()         # initialising the OUTPUT stream for writing data
    # Reading:
    ifs.SetFormat(oechem.OEFormat_MOL2) # specifying mol2 format input
    ifs.open(mol2name)           
    if oechem.OEReadMolecule(ifs,mol):
        pass
    else:
        print "Problem loading molecule!"
    # Writing:
    oechem.OETriposAtomNames(mol)           # Making the atoms unique
    ofs.SetFormat(oechem.OEFormat_PDB)      # specifying that I want a pdb format 
    ofs.open(pdbname)
    oechem.OEWriteMolecule(ofs,mol)

Convert molecules into PDBs:

In [4]:
oemol2pdb('1E.mol2',"1E.pdb")
oemol2pdb('IR783.mol2',"IR783.pdb")

Using packmol to create mixture

In [5]:
box = packmol.pack_box(["1E.pdb","IR783.pdb"], [10,3], tolerance=2.0, box_size=40)
box.save('box.pdb')


# Mixture 

tolerance 2.000000
filetype pdb
output /var/folders/_2/bwj1fkzx78v92gc1lfvf8cv589gk45/T/tmpvZyKhb.pdb
add_amber_ter


structure 1E.pdb
  number 10 
  inside box 0. 0. 0. 40.000000 40.000000 40.000000
end structure

structure IR783.pdb
  number 3 
  inside box 0. 0. 0. 40.000000 40.000000 40.000000
end structure


# Mixture 

tolerance 2.000000
filetype pdb
output /var/folders/_2/bwj1fkzx78v92gc1lfvf8cv589gk45/T/tmpvZyKhb.pdb
add_amber_ter


structure 1E.pdb
  number 10 
  inside box 0. 0. 0. 40.000000 40.000000 40.000000
end structure

structure IR783.pdb
  number 3 
  inside box 0. 0. 0. 40.000000 40.000000 40.000000
end structure



If the connect records are not present in the box of solvent, we can let openbabel infer the connections

In [6]:
#cmd = "obabel -ipdb {0}.pdb -opdb -O {1}.pdb".format("box","box_cnt")
#subprocess.call(cmd,shell=True)

This is how to generarate parameters within openmoltools without tleap

In [7]:
forcefield = app.ForceField('gaff.xml','tip3p.xml','amber99sbildn.xml')
forcefield.registerTemplateGenerator(gaffTemplateGenerator)
pdb = app.PDBFile("box.pdb")

Use modeller to add solvent and neutralise. Neutralising is the default behaviour, which means the ions added need to have parameters in the `forcefield`. This is why `amber99sbildn.xml` was added to in the previous cell.

In [8]:
modeller = app.Modeller(pdb.topology, pdb.positions)
modeller.addSolvent(forcefield)

In [9]:
system = forcefield.createSystem(pdb.topology,nonbondedMethod=app.PME, nonbondedCutoff=1.0*unit.nanometer, constraints=app.HBonds)